In [1]:
import sys
import json
sys.path.append('..')
from src.common import *
from src.analysis.model_performances import *
from copy import deepcopy
from helpers import *
import pandas as pd

EVERYTHING_POSTFIX = 'everything'

TO_PRETTY = {
    WITH_RAMIFICATIONS : 'W R',
    WITHOUT_RAMIFICATIONS : 'W/O R',
    
    'few_shot_1': 'FS-1',
    'few_shot_3': 'FS-3',
    'few_shot_5': 'FS-5',
    
    'gemma-2b': 'G-2b', 
    'gemma-7b': 'G-7b', 
    'llama2-7b-chat': 'L-7b', 
    'llama2-13b-chat': 'L-13b',
    'gemini': 'Gemini'}


In [5]:
answer_type = TRUE_FALSE_ANSWER_TYPE #FREE_ANSWER#
score_key = ACCURACY_SCORE_KEY #F1_SCORE_KEY#'accuracy'
answer_type_ext = tf_answer_type(score_key = score_key)

ids_file_name = 'dataset_ids.test'  # None
save_main_dir = f'{STATISTICS_PATH}.{ids_file_name}'
stats_all = collect_stats_all(tf_answer_type(score_key = score_key), save_main_dir=save_main_dir)
print(len(stats_all))
plan_lengths = [1,10,19]

save_dir = os.path.join(save_main_dir, 'tables', 'by_models')
os.makedirs(save_dir, exist_ok=True)

# model_prompts_combos = [('small-models', SMALL_MODELS, PROMPT_TYPES), ('big-models', BIG_MODELS, ['few_shot_1', 'few_shot_5'])]
model_prompts_combos = [('all-models', PROMPT_MODEL_NAMES, ['few_shot_1', 'few_shot_3', 'few_shot_5'])]

100%|██████████| 25920/25920 [00:03<00:00, 6930.16it/s] 


13949


In [6]:
def to_df(results_all, plan_lengths, answer_type, models=PROMPT_MODEL_NAMES,
          prompt_types = PROMPT_TYPES,
          domain = ALL_DOMAINS_KEY, subs = WITHOUT_RANDOM_SUB):
    
    index = []
    data = []    
    for plan_length in plan_lengths:
        for ramifications in RAMIFICATION_TYPES:
            index.append((plan_length, TO_PRETTY.get(ramifications, ramifications)))
            data_columns = {}
            for model_name in models:
                for prompt_type in prompt_types:
                    res_obj = filter_single_selector(results_all, plan_length, ALL_QUESTION_CATEGORIES_KEY, ramifications, model_name, prompt_type, domain, answer_type, subs)
                    # print(res_obj)
                    if res_obj:
                        mean = res_obj['result']
                        sem = None
                        if res_obj['result_other']:
                            sem = res_obj['result_other']['sem']
                        not_corrupted = res_obj['stats']['num_not_corrupted']
                        final_res = (mean, sem, not_corrupted)
                    else:
                        final_res = (None, None, None)
                    final_res = tuple([round(v*100, 2) if v else v for v in final_res ])
                    final_res = '${'+str(final_res[0])+'}_{'+str(final_res[1])+'}$'
                    data_columns[(TO_PRETTY.get(model_name, model_name), TO_PRETTY.get(prompt_type, prompt_type))] = final_res
            data.append(data_columns)
    return pd.DataFrame(data, index = index)

def to_df_by_category(results_all, answer_type,  
                      model_names = PROMPT_MODEL_NAMES,
                      prompt_types= PROMPT_TYPES,
                      ramifications = WITHOUT_RAMIFICATIONS,
                      domain = ALL_DOMAINS_KEY, 
                      subs = WITHOUT_RANDOM_SUB,
                      plan_length=19):

    index = []
    data = []    
    for question_category in QUESTION_CATEGORIES[:-1]:
        index.append(question_category)
        data_columns = {}
        for model_name in model_names:
            for prompt_type in prompt_types:
                res_obj = filter_single_selector(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain, answer_type, subs)
                if res_obj:
                    mean = res_obj['result']
                    sem = None
                    if res_obj['result_other']:
                        sem = res_obj['result_other']['sem']
                    not_corrupted = res_obj['stats']['num_not_corrupted']
                    final_res = (mean, sem, not_corrupted)
                else:
                    final_res = (None, None, None)
                final_res = tuple([round(v*100, 2) if v else v for v in final_res ])
                final_res = '${'+str(final_res[0])+'}_{'+str(final_res[1])+'}$'
                data_columns[(TO_PRETTY.get(model_name,model_name), TO_PRETTY.get(prompt_type,prompt_type))] = final_res
        data.append(data_columns)
    return pd.DataFrame(data, index = index)

def df_to_latex_table(df):
        latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)
        return latex_table[latex_table.find('\midrule'):].replace("bottomrule", 'crap').replace("\crap", '').replace("\end{tabular}", '')

def assemble_table(results_all, answer_type, domain, score_key=None):
    latex_table_all = ''
    with open('latex_table_template/top') as f:
        latex_table_all += f.read() + '\n'
    latex_table_all += '\n'.join([df_to_latex_table(to_df(results_all, answer_type, plan_length, domain)) for plan_length in PLAN_LENGTHS])
    with open('latex_table_template/bottom') as f:
        latex_table_all += f.read()
    
    caption = f'{answer_type}, {score_key} scores for {domain}'.replace('_', ' ')
    latex_table_all = latex_table_all.replace('REPLACE_CAPTION_KEY', caption)
    
    return latex_table_all


In [7]:
for subs in [WITHOUT_RANDOM_SUB, WITH_RANDOM_SUB]:
    for model_save_name, model_names, prompt_types in model_prompts_combos:
        df = to_df(stats_all, plan_lengths, answer_type, prompt_types=prompt_types, models=model_names, subs=subs)
        print(df)
        
        caption_nl = f'performance of {model_save_name} on the test set, {subs}'.replace('_', ' ')
        latex_table = to_latex_table(df, caption_nl, label=model_save_name)
        save_key = f'all.{model_save_name}.{subs}'
        with open(os.path.join(save_dir, f'{save_key}.tex'), 'w') as f:
            f.write(latex_table)

                 (G-2b, FS-1)      (G-2b, FS-3)      (G-2b, FS-5)  \
(1, W R)     ${46.62}_{1.14}$  ${41.73}_{1.19}$  ${36.21}_{1.23}$   
(1, W/O R)   ${46.24}_{1.18}$  ${41.15}_{1.17}$  ${36.46}_{1.23}$   
(10, W R)    ${45.24}_{1.14}$   ${38.1}_{1.16}$  ${33.27}_{1.21}$   
(10, W/O R)  ${43.65}_{1.17}$  ${36.58}_{1.14}$   ${33.16}_{1.2}$   
(19, W R)    ${45.17}_{1.15}$  ${36.98}_{1.15}$  ${28.22}_{1.21}$   
(19, W/O R)  ${44.66}_{1.18}$   ${36.0}_{1.15}$  ${27.93}_{1.21}$   

                 (G-7b, FS-1)      (G-7b, FS-3)      (G-7b, FS-5)  \
(1, W R)     ${51.59}_{1.18}$   ${47.67}_{1.2}$  ${53.92}_{1.28}$   
(1, W/O R)   ${51.28}_{1.16}$   ${None}_{None}$   ${None}_{None}$   
(10, W R)    ${51.35}_{1.18}$  ${53.91}_{1.18}$  ${60.09}_{1.27}$   
(10, W/O R)  ${50.81}_{1.16}$   ${None}_{None}$   ${None}_{None}$   
(19, W R)    ${49.18}_{1.19}$  ${56.24}_{1.19}$  ${56.91}_{1.34}$   
(19, W/O R)  ${48.08}_{1.16}$   ${None}_{None}$   ${None}_{None}$   

                 (L-7b, FS-1)   

/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:21: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)


                 (G-2b, FS-1)      (G-2b, FS-3)      (G-2b, FS-5)  \
(1, W R)     ${45.67}_{1.27}$  ${45.14}_{1.58}$  ${44.06}_{2.47}$   
(1, W/O R)    ${45.7}_{1.26}$  ${45.13}_{1.68}$  ${40.85}_{2.38}$   
(10, W R)    ${45.32}_{1.32}$  ${44.89}_{2.09}$  ${50.0}_{35.36}$   
(10, W/O R)   ${44.05}_{1.3}$  ${46.02}_{2.22}$   ${100.0}_{0.0}$   
(19, W R)    ${46.08}_{1.45}$  ${32.84}_{4.06}$   ${None}_{None}$   
(19, W/O R)  ${43.84}_{1.43}$  ${43.48}_{4.62}$   ${None}_{None}$   

                 (G-7b, FS-1)      (G-7b, FS-3)      (G-7b, FS-5)  \
(1, W R)     ${49.19}_{1.18}$  ${52.79}_{1.24}$  ${56.94}_{1.42}$   
(1, W/O R)   ${48.76}_{1.19}$    ${52.5}_{1.2}$  ${55.32}_{1.43}$   
(10, W R)    ${50.69}_{1.17}$   ${54.1}_{1.28}$  ${57.71}_{1.53}$   
(10, W/O R)  ${49.53}_{1.18}$  ${53.53}_{1.23}$  ${56.55}_{1.54}$   
(19, W R)    ${48.43}_{1.18}$  ${55.22}_{1.37}$  ${53.82}_{1.93}$   
(19, W/O R)  ${48.87}_{1.19}$   ${53.58}_{1.3}$  ${53.31}_{1.96}$   

                 (L-7b, FS-1)   

/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:21: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)


# Plot By Category

In [11]:
plan_length = 19
for subs in [WITHOUT_RANDOM_SUB, WITH_RANDOM_SUB]:
    for model_save_name, model_names, prompt_types in model_prompts_combos:
        df2 = to_df_by_category(stats_all, answer_type, model_names=model_names, prompt_types=prompt_types, subs=subs)
        print(df2)
        
        caption_nl = f'performance of {model_save_name} on the test set by categories, {subs}, pl-{plan_length}'
        save_key = f'by_categories.{model_save_name}.{subs}'
        
        latex_table_all = to_latex_table(df2, caption_nl, label=save_key)
        # with open(os.path.join(save_dir, f'{save_key}.tex'), 'w') as f:
        #     f.write(latex_table_all)

                          (G-2b, FS-1)      (G-2b, FS-3)      (G-2b, FS-5)  \
object_tracking       ${40.38}_{2.13}$  ${35.81}_{2.09}$  ${28.41}_{2.15}$   
fluent_tracking        ${43.2}_{1.97}$  ${30.71}_{1.85}$  ${19.58}_{1.82}$   
state_tracking        ${47.73}_{7.53}$   ${4.55}_{3.14}$     ${0.0}_{0.0}$   
action_executability  ${53.66}_{5.51}$  ${43.37}_{5.44}$  ${32.79}_{6.01}$   
effects               ${49.54}_{2.76}$  ${42.14}_{2.77}$  ${35.29}_{2.99}$   
numerical_reasoning   ${45.78}_{5.47}$   ${45.88}_{5.4}$  ${38.24}_{5.89}$   
hallucination         ${52.44}_{5.52}$  ${53.95}_{5.72}$  ${52.63}_{6.61}$   

                          (G-7b, FS-1)     (G-7b, FS-3)     (G-7b, FS-5)  \
object_tracking       ${47.08}_{2.13}$  ${None}_{None}$  ${None}_{None}$   
fluent_tracking       ${49.39}_{1.95}$  ${None}_{None}$  ${None}_{None}$   
state_tracking        ${59.09}_{7.41}$  ${None}_{None}$  ${None}_{None}$   
action_executability  ${53.57}_{5.44}$  ${None}_{None}$  ${None}_{None}

/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:21: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)


                          (G-2b, FS-1)       (G-2b, FS-3)     (G-2b, FS-5)  \
object_tracking       ${42.82}_{2.46}$   ${31.58}_{7.54}$  ${None}_{None}$   
fluent_tracking       ${41.44}_{2.55}$  ${53.33}_{12.88}$  ${None}_{None}$   
state_tracking         ${None}_{None}$    ${None}_{None}$  ${None}_{None}$   
action_executability  ${38.71}_{6.19}$  ${33.33}_{15.71}$  ${None}_{None}$   
effects               ${47.21}_{3.27}$  ${47.83}_{10.42}$  ${None}_{None}$   
numerical_reasoning   ${47.83}_{6.01}$   ${57.14}_{10.8}$  ${None}_{None}$   
hallucination         ${53.33}_{6.44}$  ${44.44}_{16.56}$  ${None}_{None}$   

                          (G-7b, FS-1)      (G-7b, FS-3)      (G-7b, FS-5)  \
object_tracking       ${45.45}_{2.17}$  ${48.53}_{2.29}$  ${51.41}_{3.17}$   
fluent_tracking        ${49.36}_{2.0}$   ${50.1}_{2.22}$  ${48.31}_{3.75}$   
state_tracking        ${52.27}_{7.53}$   ${None}_{None}$   ${None}_{None}$   
action_executability   ${50.0}_{5.59}$   ${55.26}_{5.7}$  ${57.

/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:21: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)
